# LUZ Rollup

In [8]:
import os 
import pandas as pd
import numpy as np
import glob
import pyodbc
import gc

In [9]:
# LUZ Crosswalk 
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                    'Server=sql2014b8.sandag.org;'
                    'Database=GeoDepot;'
                    'Trusted_Connection=yes;')

with open(r'C:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2023\2023-024 Estimates 2022\series_14_automation\sql_queries\luz_rollup.sql', 'r') as sql_file:
    sql_query = sql_file.read()

df =  pd.read_sql_query(sql_query, conn)
df

,MGRA,LUZ
0,1,95
1,2,95
2,3,95
3,4,95
4,5,95
...,...,...
22997,22998,14
22998,22999,14
22999,23000,14
23000,23001,14


In [21]:
categories = ['age', 'ethnicity', 'household_income', 'households', 'housing', 'population', 'sex']


In [4]:
def create_LUZ_output_for_2021_01(category, to_csv):
    data = pd.read_csv(rf'C:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2023\2023-024 Estimates 2022\series_14_automation\files\QA_2021_01_mgra_{category}.csv')

    output = data.merge(df, how='left', left_on='mgra', right_on='MGRA')
    output = output.drop(['mgra', 'MGRA'], axis=1)
    output = output.sort_values(['LUZ', 'yr_id'])
    output = output.groupby(['LUZ', 'yr_id']).sum()
    
    if to_csv == True:
        output.reset_index().to_csv(rf'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-023 Estimates 2022\Results\Test 5- Vintage Comparison\2021_01\{category}\QA_2021_01_LUZ_{category}.csv', index=False)

    return output
    

In [5]:
def create_LUZ_output_for_age_sex_eth_2021_01(category, to_csv):
    data = pd.read_csv(rf'C:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2023\2023-024 Estimates 2022\series_14_automation\files\QA_2021_01_mgra_{category}.csv')

    output = data.merge(df, how='left', left_on='mgra', right_on='MGRA')
    output = output.drop(['mgra', 'MGRA'], axis=1)
    output = output.sort_values(['LUZ', 'yr_id'])
    output = output.groupby(['LUZ', 'yr_id', 'age_group', 'sex']).sum()
    
    if to_csv == True:
        output.reset_index().to_csv(rf'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-023 Estimates 2022\Results\Test 5- Vintage Comparison\2021_01\{category}\QA_2021_01_LUZ_{category}.csv', index=False)

    return output

In [28]:
for category in categories:
    create_LUZ_output_for_2021_01(category=category, to_csv=True)
    print(f"{category} is done.")

age is done.
ethnicity is done.
household_income is done.
households is done.
housing is done.
population is done.
sex is done.


In [7]:
create_LUZ_output_for_2021_01(category="households", to_csv=True)

Family household:female householder, no husband present   \
LUZ yr_id                                                             
1   2020                                                1261          
    2021                                                1251          
2   2020                                                1977          
    2021                                                1992          
3   2020                                                3670          
...                                                      ...          
227 2021                                                  28          
228 2020                                                   3          
    2021                                                   3          
229 2020                                                 335          
    2021                                                 335          

           Family household:male householder, no wife present  \
LUZ yr_id                                                       
1   2020                                                2766    
    2021                                                2843    
2   2020                                                3333    
    2021                                                3450    
3   2020                                                6067    
...                                                      ...    
227 2021                                                  41    
228 2020                                                   5    
    2021                                                   4    
229 2020                                                 331    
    2021                                                 358    

           Family household:married-couple  \
LUZ yr_id                                    
1   2020                              2337   
    2021                              2316   
2   2020                              2088   
    2021                              2077   
3   2020                              3744   
...                                    ...   
227 2021                                14   
228 2020                                 4   
    2021                                 4   
229 2020                               114   
    2021                               121   

           Nonfamily household:female householder, living alone  \
LUZ yr_id                                                         
1   2020                                                 295      
    2021                                                 272      
2   2020                                                 454      
    2021                                                 440      
3   2020                                                 928      
...                                                      ...      
227 2021                                                  11      
228 2020                                                   5      
    2021                                                   2      
229 2020                                                 164      
    2021                                                 150      

           Nonfamily household:female householder, not living alone  \
LUZ yr_id                                                             
1   2020                                                 235          
    2021                                                 234          
2   2020                                                 282          
    2021                                                 291          
3   2020                                                 521          
...                                                      ...          
227 2021                                                  12          
228 2020                                                   3          
    2021                                                   5

In [6]:
create_LUZ_output_for_age_sex_eth_2021_01(category='age_sex_ethnicity', to_csv=True)

Hispanic  \
LUZ yr_id age_group    sex                
1   2020  10 to 14     Female       343   
                       Male         314   
          15 to 17     Female       210   
                       Male         249   
          18 and 19    Female       101   
...                                 ...   
229 2021  80 to 84     Male          19   
          85 and Older Female        35   
                       Male          20   
          Under 5      Female       158   
                       Male         195   

                               Non-Hispanic, American Indian or Alaska Native  \
LUZ yr_id age_group    sex                                                      
1   2020  10 to 14     Female                                               5   
                       Male                                                 3   
          15 to 17     Female                                               1   
                       Male                                                 2   
          18 and 19    Female                                               1   
...                                                                       ...   
229 2021  80 to 84     Male                                                 0   
          85 and Older Female                                               0   
                       Male                                                 0   
          Under 5      Female                                               1   
                       Male                                                 1   

                               Non-Hispanic, Asian  Non-Hispanic, Black  \
LUZ yr_id age_group    sex                                                
1   2020  10 to 14     Female                   30                   16   
                       Male                     24                   23   
          15 to 17     Female                    8                    8   
                       Male                     18                   19   
          18 and 19    Female                    5                    8   
...                                            ...                  ...   
229 2021  80 to 84     Male                      2                    0   
          85 and Older Female                    3                    1   
                       Male                      2                    4   
          Under 5      Female                    7                    6   
                       Male                      6                    9   

                               Non-Hispanic, Hawaiian or Pacific Islander  \
LUZ yr_id age_group    sex                                                  
1   2020  10 to 14     Female                                           0   
                       Male                                             5   
          15 to 17     Female                                           3   
                       Male                                             2   
          18 and 19    Female                                           1   
...                                                                   ...   
229 2021  80 to 84     Male                                             1   
          85 and Older Female                                           1   
                       Male                                             0   
          Under 5      Female                                           0   
                       Male                                             0   

                               Non-Hispanic, Other  \
LUZ yr_id age_group    sex                           
1   2020  10 to 14     Female                    2   
                       Male                      2   
          15 to 17     Female                    2   
                       Male                      1   
          18 and 19    Female                    2   
...                                            ...   
229 202

In [10]:
data = pd.read_csv(r'C:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2023\2023-024 Estimates 2022\series_14_automation\files\QA_2021_01_mgra_population.csv')
data

,mgra,yr_id,Total Population,Household Population,Group Quarters - Military,Group Quarters - College,Group Quarters - Other
0,1,2010,41,41,0,0,0
1,1,2011,41,41,0,0,0
2,1,2012,42,42,0,0,0
3,1,2013,44,44,0,0,0
4,1,2014,47,47,0,0,0
...,...,...,...,...,...,...,...
276019,23002,2017,0,0,0,0,0
276020,23002,2018,0,0,0,0,0
276021,23002,2019,0,0,0,0,0
276022,23002,2020,0,0,0,0,0


In [15]:
output = data.merge(df, how='left', left_on='mgra', right_on='MGRA')
output = output.drop(['mgra', 'MGRA'], axis=1)
output = output.sort_values(['LUZ', 'yr_id'])
output = output.groupby(['LUZ', 'yr_id']).sum()
output

Total Population  Household Population  Group Quarters - Military  \
LUZ yr_id                                                                      
1   2010              18400                 18154                          0   
    2011              18936                 18690                          0   
    2012              19752                 19506                          0   
    2013              20701                 20455                          0   
    2014              20970                 20724                          0   
...                     ...                   ...                        ...   
229 2017               6261                  6261                          0   
    2018               6287                  6287                          0   
    2019               6250                  6250                          0   
    2020               6064                  6064                          0   
    2021               5982                  5982                          0   

           Group Quarters - College  Group Quarters - Other  
LUZ yr_id                                                    
1   2010                          0                     246  
    2011                          0                     246  
    2012                          0                     246  
    2013                          0                     246  
    2014                          0                     246  
...                             ...                     ...  
229 2017                          0                       0  
    2018                          0                       0  
    2019                          0                       0  
    2020                          0                       0  
    2021                          0                       0  

[2748 rows x 5 columns]

: 